# Inspect observation_spec()

本 notebook 用于在当前环境中打印 `SC2Env.observation_spec()` 以便查看观测空间结构。

说明：如果本地未安装或未配置 StarCraft II / pysc2，创建环境会失败，这个单元包含异常处理以便给出可读的错误信息。

In [1]:
# 尝试创建一个最小的 SC2Env 并打印 observation_spec()。
# 该单元包含异常处理，若环境不可用会打印错误和回溯。
from absl import flags
# 确保先导入 run_configs 以注册 sc2_run_config 等 flags，再解析它们
import pysc2.run_configs
# 解析 absl flags 并设置 sc2_run_config（Windows 平台使用 'Windows'）
try:
    flags.FLAGS(['notebook', '--sc2_run_config=Windows'])
except Exception:
    # 如果 flags 已解析或其它异常，忽略并继续（后续导入会给出更具体错误）
    pass
try:
    from pysc2.env import sc2_env
    from pysc2.lib import features
    aif = sc2_env.AgentInterfaceFormat(
        feature_dimensions=sc2_env.Dimensions(screen=64, minimap=64),
        use_feature_units=True,
        use_raw_units=True,
        use_unit_counts=True,
        use_camera_position=True,
    )
    env = sc2_env.SC2Env(
        map_name="MoveToBeacon",
        players=[sc2_env.Agent(sc2_env.Race.terran)],
        agent_interface_format=aif,
        step_mul=8,
        game_steps_per_episode=0,
        visualize=False,
    )
    try:
        spec = env.observation_spec()
        # 如果是 dict，逐项打印以便阅读
        if isinstance(spec, dict):
            import pprint
            pp = pprint.PrettyPrinter(width=120)
            pp.pprint(spec)
        else:
            print(spec)
    finally:
        try:
            env.close()
        except Exception:
            pass
except Exception as e:
    print("Failed to import or create SC2Env:", e)
    import traceback
    traceback.print_exc()

c:\Users\ydx\.conda\envs\PPO-minigame\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.10.19)
Hello from the pygame community. https://www.pygame.org/contribute.html
({'action_result': (0,), 'alerts': (0,), 'build_queue': (0, 7), 'cargo': (0, 7), 'cargo_slots_available': (1,), 'control_groups': (10, 2), 'game_loop': (1,), 'last_actions': (0,), 'map_name': (0,), 'multi_select': (0, 7), 'player': (11,), 'production_queue': (0, 2), 'score_cumulative': (13,), 'score_by_category': (11, 5), 'score_by_vital': (3, 3), 'single_select': (0, 7), 'available_actions': (0,), 'feature_screen': (27, 64, 64), 'feature_minimap': (11, 64, 64), 'feature_units': (0, 46), 'feature_effects': (0, 6), 'raw_units': (0, 46), 'raw_effects': (0, 6), 'radar': (0, 3), 'upgrades': (0,), 'unit_counts': (0, 2), 'camera_position': (2,), 'camera_size': (2,), 'home_race_requested': (1,), 'away_race_requested': (1,)},)


In [7]:
# 实验：检查“通用字段”在当前 observation 中是否都出现
from absl import flags
import pysc2.run_configs
try:
    flags.FLAGS(['notebook', '--sc2_run_config=Windows'])
except Exception:
    pass
try:
    from pysc2.env import sc2_env
    aif = sc2_env.AgentInterfaceFormat(
        feature_dimensions=sc2_env.Dimensions(screen=64, minimap=64),
        use_feature_units=True,
        use_raw_units=True,
        use_unit_counts=True,
        use_camera_position=True,
    )
    env = sc2_env.SC2Env(
        map_name="MoveToBeacon",
        players=[sc2_env.Agent(sc2_env.Race.terran)],
        agent_interface_format=aif,
        step_mul=8,
        game_steps_per_episode=0,
        visualize=False,
    )
    try:
        timesteps = env.reset()
        obs = timesteps[0].observation
        common_fields = [
            "action_result",
            "alerts",
            "build_queue",
            "cargo",
            "cargo_slots_available",
            "control_groups",
            "game_loop",
            "last_actions",
            "map_name",
            "multi_select",
            "player",
            "production_queue",
            "score_cumulative",
            "score_by_category",
            "score_by_vital",
            "single_select",
            "upgrades",
            "home_race_requested",
            "away_race_requested",
        ]
        present = [k for k in common_fields if k in obs]
        missing = [k for k in common_fields if k not in obs]
        print("present common fields:", present)
        print("missing common fields:", missing)
    finally:
        try:
            env.close()
        except Exception:
            pass
except Exception as e:
    print("Failed to run common-fields check:", e)
    import traceback
    traceback.print_exc()

present common fields: ['action_result', 'alerts', 'build_queue', 'cargo', 'cargo_slots_available', 'control_groups', 'game_loop', 'last_actions', 'map_name', 'multi_select', 'player', 'production_queue', 'score_cumulative', 'score_by_category', 'score_by_vital', 'single_select', 'upgrades', 'home_race_requested', 'away_race_requested']
missing common fields: []


In [8]:
# 生成 leader_unit_type / unit_type 的 id -> name 映射（取样输出以验证）
from absl import flags
import pysc2.run_configs
try:
    flags.FLAGS(['notebook', '--sc2_run_config=Windows'])
except Exception:
    pass
try:
    from pysc2.env import sc2_env
    aif = sc2_env.AgentInterfaceFormat(
        feature_dimensions=sc2_env.Dimensions(screen=64, minimap=64),
    )
    env = sc2_env.SC2Env(
        map_name="MoveToBeacon",
        players=[sc2_env.Agent(sc2_env.Race.terran)],
        agent_interface_format=aif,
        step_mul=8,
        game_steps_per_episode=0,
        visualize=False,
    )
    try:
        data = env._controllers[0].data_raw()  # ResponseData
        unit_id_to_name = {u.unit_id: u.name for u in data.units}
        print("unit_id_to_name size:", len(unit_id_to_name))
        # 打印前 20 个（按 id 排序）
        for unit_id in sorted(unit_id_to_name)[:20]:
            print(unit_id, unit_id_to_name[unit_id])
    finally:
        try:
            env.close()
        except Exception:
            pass
except Exception as e:
    print("Failed to generate unit_id_to_name mapping:", e)
    import traceback
    traceback.print_exc()

unit_id_to_name size: 2005
0 
1 System_Snapshot_Dummy
2 Ball
3 StereoscopicOptionsUnit
4 Colossus
5 TechLab
6 Reactor
7 InfestorTerran
8 BanelingCocoon
9 Baneling
10 Mothership
11 PointDefenseDrone
12 Changeling
13 ChangelingZealot
14 ChangelingMarineShield
15 ChangelingMarine
16 ChangelingZerglingWings
17 ChangelingZergling
18 CommandCenter
19 SupplyDepot


In [13]:
# 试验：按附录 F 的方法从 ResponseData 中提取 ability_id -> name 映射（不导出 CSV）
from absl import flags
import pysc2.run_configs
try:
    flags.FLAGS(['notebook', '--sc2_run_config=Windows'])
except Exception:
    pass
try:
    from pysc2.env import sc2_env
    aif = sc2_env.AgentInterfaceFormat(
        feature_dimensions=sc2_env.Dimensions(screen=64, minimap=64),
    )
    env = sc2_env.SC2Env(
        map_name="MoveToBeacon",
        players=[sc2_env.Agent(sc2_env.Race.terran)],
        agent_interface_format=aif,
        step_mul=8,
        game_steps_per_episode=0,
        visualize=False,
    )
    try:
        data = env._controllers[0].data_raw()  # ResponseData
        abilities = []
        if hasattr(data, 'abilities'):
            abilities = list(data.abilities)
        elif hasattr(data, 'abilities_data'):
            abilities = list(data.abilities_data)
        else:
            print('ResponseData 属性列表:', [f for f in dir(data) if not f.startswith('_')])
            abilities = []

        ability_id_to_name = {}
        for a in abilities:
            aid = getattr(a, 'ability_id', None)
            name = getattr(a, 'link_name', '') or getattr(a, 'button_name', '')
            if not name:
                try:
                    name = str(a)
                except Exception:
                    name = ''
            if aid is not None:
                ability_id_to_name[aid] = name

        print('ability_id_to_name size:', len(ability_id_to_name))
        # 打印前 40 个样例（按 id 排序）
        for aid in sorted(ability_id_to_name)[:40]:
            print(aid, ability_id_to_name[aid])

        # === 重点：对比同名 ability 的字段差异 ===
        target_name = 'attack'  # 可改为你关心的名字，例如 'move'
        same_name = [a for a in abilities if (getattr(a, 'link_name', '') or getattr(a, 'button_name', '')) == target_name]
        print('\nSame-name abilities for:', target_name, 'count=', len(same_name))
        if len(same_name) == 0:
            print('No abilities matched name:', target_name)
        else:
            # 收集每条 ability 的字段 dict，用于比较差异
            def to_dict(obj):
                try:
                    return {fd.name: val for fd, val in obj.ListFields()}
                except Exception:
                    return {}
            dicts = [to_dict(a) for a in same_name]
            keys = sorted({k for d in dicts for k in d.keys()})
            # 输出每条 ability 的关键字段摘要
            for a, d in zip(same_name, dicts):
                print('ability_id:', getattr(a, 'ability_id', None), 'link_index:', d.get('link_index'), 'button_name:', d.get('button_name'), 'hotkey:', d.get('hotkey'), 'target:', d.get('target'))
            # 逐字段比较是否存在差异
            print('\nField differences:')
            for k in keys:
                values = [d.get(k, None) for d in dicts]
                if len(set(map(str, values))) > 1:
                    print(k, '->', values)
    finally:
        try:
            env.close()
        except Exception:
            pass
except Exception as e:
    print('Failed to generate ability mapping:', e)
    import traceback; traceback.print_exc()

ability_id_to_name size: 4134
0 Null
1 Smart
2 Taunt
3 Taunt
4 stop
5 stop
6 stop
7 stop
8 stop
9 stop
10 HoldFire
11 HoldFire
12 HoldFire
13 HoldFire
14 HoldFire
15 HoldFire
16 move
17 move
18 move
19 move
20 move
21 Beacon
22 Beacon
23 attack
24 attack
25 attack
26 SprayTerran
27 SprayTerran
28 SprayZerg
29 SprayZerg
30 SprayProtoss
31 SprayProtoss
32 SalvageShared
33 SalvageShared
34 Corruption
35 Corruption
36 GhostHoldFire
37 GhostHoldFire
38 GhostWeaponsFree
39 GhostWeaponsFree

Same-name abilities for: attack count= 3
ability_id: 23 link_index: 0 button_name: Attack hotkey: A target: 4
ability_id: 24 link_index: 1 button_name: AttackTowards hotkey:  target: 2
ability_id: 25 link_index: 2 button_name: AttackBarrage hotkey:  target: 2

Field differences:
ability_id -> [23, 24, 25]
button_name -> ['Attack', 'AttackTowards', 'AttackBarrage']
friendly_name -> ['Attack Attack', 'Attack AttackTowards', 'Attack AttackBarrage']
hotkey -> ['A', '', '']
link_index -> [0, 1, 2]
remaps_to_ab

In [10]:
# 调试：打印 abilities 第一个条目的字段以确定正确的属性名
from absl import flags
import pysc2.run_configs
try:
    flags.FLAGS(['notebook', '--sc2_run_config=Windows'])
except Exception:
    pass
try:
    from pysc2.env import sc2_env
    aif = sc2_env.AgentInterfaceFormat(feature_dimensions=sc2_env.Dimensions(screen=64, minimap=64))
    env = sc2_env.SC2Env(map_name="MoveToBeacon", players=[sc2_env.Agent(sc2_env.Race.terran)], agent_interface_format=aif, step_mul=8, game_steps_per_episode=0, visualize=False)
    try:
        data = env._controllers[0].data_raw()
        if hasattr(data, 'abilities'):
            print('abilities count:', len(data.abilities))
            if len(data.abilities) > 0:
                a = data.abilities[0]
                print('ability item type:', type(a))
                try:
                    for fd, val in a.ListFields():
                        print(fd.name, '->', val)
                except Exception:
                    print('dir(a):', [f for f in dir(a) if not f.startswith('_')])
                    print(repr(a))
        else:
            print('ResponseData fields:', [f for f in dir(data) if not f.startswith('_')])
    finally:
        try:
            env.close()
        except Exception:
            pass
except Exception as e:
    print('inspect failed:', e)
    import traceback; traceback.print_exc()

abilities count: 4134
ability item type: <class 's2clientprotocol.data_pb2.AbilityData'>
ability_id -> 0
link_name -> Null
link_index -> 255
button_name -> 
available -> True


In [4]:
# 列举 s2clientprotocol 中的 ActionResult 枚举（值域与名字）
try:
    from s2clientprotocol import error_pb2 as sc_err
    enum_desc = sc_err.ActionResult.DESCRIPTOR
    print("ActionResult enum values (number -> name):")
    for v in enum_desc.values:
        print(v.number, v.name)
except Exception as e:
    print("Failed to enumerate ActionResult enum:", e)
    import traceback; traceback.print_exc()

ActionResult enum values (number -> name):
1 Success
2 NotSupported
3 Error
4 CantQueueThatOrder
5 Retry
6 Cooldown
7 QueueIsFull
8 RallyQueueIsFull
9 NotEnoughMinerals
10 NotEnoughVespene
11 NotEnoughTerrazine
12 NotEnoughCustom
13 NotEnoughFood
14 FoodUsageImpossible
15 NotEnoughLife
16 NotEnoughShields
17 NotEnoughEnergy
18 LifeSuppressed
19 ShieldsSuppressed
20 EnergySuppressed
21 NotEnoughCharges
22 CantAddMoreCharges
23 TooMuchMinerals
24 TooMuchVespene
25 TooMuchTerrazine
26 TooMuchCustom
27 TooMuchFood
28 TooMuchLife
29 TooMuchShields
30 TooMuchEnergy
31 MustTargetUnitWithLife
32 MustTargetUnitWithShields
33 MustTargetUnitWithEnergy
34 CantTrade
35 CantSpend
36 CantTargetThatUnit
37 CouldntAllocateUnit
38 UnitCantMove
39 TransportIsHoldingPosition
40 BuildTechRequirementsNotMet
41 CantFindPlacementLocation
42 CantBuildOnThat
43 CantBuildTooCloseToDropOff
44 CantBuildLocationInvalid
45 CantSeeBuildLocation
46 CantBuildTooCloseToCreepSource
47 CantBuildTooCloseToResources
48 Cant

In [5]:
# 列举 s2clientprotocol 中的 Alert 枚举（值域与名字）
try:
    from s2clientprotocol import sc2api_pb2 as sc_pb
    enum_desc = sc_pb.Alert.DESCRIPTOR
    print("Alert enum values (number -> name):")
    for v in enum_desc.values:
        print(v.number, v.name)
except Exception as e:
    print("Failed to enumerate Alert enum:", e)
    import traceback; traceback.print_exc()

Alert enum values (number -> name):
3 AlertError
4 AddOnComplete
5 BuildingComplete
6 BuildingUnderAttack
7 LarvaHatched
8 MergeComplete
9 MineralsExhausted
10 MorphComplete
11 MothershipComplete
12 MULEExpired
1 NuclearLaunchDetected
13 NukeComplete
2 NydusWormDetected
14 ResearchComplete
15 TrainError
16 TrainUnitComplete
17 TrainWorkerComplete
18 TransformationComplete
19 UnitUnderAttack
20 UpgradeComplete
21 VespeneExhausted
22 WarpInComplete


In [14]:
# 验证：从 ResponseData 提取 upgrades 映射的可行性（仅打印示例，不写文件）
from absl import flags
import pysc2.run_configs
try:
    flags.FLAGS(['notebook', '--sc2_run_config=Windows'])
except Exception:
    pass
try:
    from pysc2.env import sc2_env
    aif = sc2_env.AgentInterfaceFormat(feature_dimensions=sc2_env.Dimensions(screen=64, minimap=64))
    env = sc2_env.SC2Env(map_name="MoveToBeacon", players=[sc2_env.Agent(sc2_env.Race.terran)], agent_interface_format=aif, step_mul=8, game_steps_per_episode=0, visualize=False)
    try:
        data = env._controllers[0].data_raw()  # ResponseData
        # 有些版本字段名为 'upgrades' 或 'upgrades_data'，兼容处理
        upgrades = getattr(data, 'upgrades', None) or getattr(data, 'upgrades_data', None) or []
        upgrade_map = {u.upgrade_id: u.name for u in upgrades}
        print('sampled upgrades count:', len(upgrade_map))
        for uid in sorted(upgrade_map)[:50]:
            print(uid, upgrade_map[uid])
    finally:
        try:
            env.close()
        except Exception:
            pass
except Exception as e:
    print('Failed to sample upgrades mapping:', e)
    import traceback; traceback.print_exc()

sampled upgrades count: 306
0 
1 CarrierLaunchSpeedUpgrade
2 GlialReconstitution
3 TunnelingClaws
4 ChitinousPlating
5 HiSecAutoTracking
6 TerranBuildingArmor
7 TerranInfantryWeaponsLevel1
8 TerranInfantryWeaponsLevel2
9 TerranInfantryWeaponsLevel3
10 NeosteelFrame
11 TerranInfantryArmorsLevel1
12 TerranInfantryArmorsLevel2
13 TerranInfantryArmorsLevel3
14 ReaperSpeed
15 Stimpack
16 ShieldWall
17 PunisherGrenades
18 SiegeTech
19 HighCapacityBarrels
20 BansheeCloak
21 MedivacCaduceusReactor
22 RavenCorvidReactor
23 HunterSeeker
24 DurableMaterials
25 PersonalCloaking
26 GhostMoebiusReactor
27 TerranVehicleArmorsLevel1
28 TerranVehicleArmorsLevel2
29 TerranVehicleArmorsLevel3
30 TerranVehicleWeaponsLevel1
31 TerranVehicleWeaponsLevel2
32 TerranVehicleWeaponsLevel3
33 TerranShipArmorsLevel1
34 TerranShipArmorsLevel2
35 TerranShipArmorsLevel3
36 TerranShipWeaponsLevel1
37 TerranShipWeaponsLevel2
38 TerranShipWeaponsLevel3
39 ProtossGroundWeaponsLevel1
40 ProtossGroundWeaponsLevel2
41 Proto

In [18]:
# Demo：创建带指定请求种族的环境并打印 home/away requested race（仅打印示例，不写文件）
from absl import flags
import pysc2.run_configs
try:
    flags.FLAGS(['notebook', '--sc2_run_config=Windows'])
except Exception:
    pass
try:
    from pysc2.env import sc2_env
    # Race enum is provided in common_pb2 in many s2clientprotocol versions
    try:
        from s2clientprotocol import common_pb2 as sc_common
    except Exception:
        sc_common = None
    # prepare agent interface and players
    aif = sc2_env.AgentInterfaceFormat(feature_dimensions=sc2_env.Dimensions(screen=64, minimap=64))
    players = [sc2_env.Agent(sc2_env.Race.terran, name='agent_terran'), sc2_env.Agent(sc2_env.Race.zerg, name='agent_zerg')]
    env = sc2_env.SC2Env(map_name="Simple64", players=players, agent_interface_format=aif, step_mul=8, game_steps_per_episode=0, visualize=False)
    try:
        timesteps = env.reset()
        obs = timesteps[0].observation
        # Build race_map from common_pb2 if available, otherwise try sc2api_pb2 as a fallback
        race_map = {}
        if sc_common is not None:
            try:
                race_map = {v.number: v.name for v in sc_common.Race.DESCRIPTOR.values}
            except Exception:
                race_map = {}
        if not race_map:
            try:
                from s2clientprotocol import sc2api_pb2 as sc_pb
                race_map = {v.number: v.name for v in sc_pb.Race.DESCRIPTOR.values}
            except Exception:
                race_map = {}
        # helper: robustly extract an integer from observation field
        def _obs_field_to_int(field):
            try:
                # sequence-like (e.g., array or list with one element)
                if hasattr(field, '__len__') and len(field) > 0:
                    return int(field[0])
                return int(field)
            except Exception:
                try:
                    import numpy as _np
                    arr = _np.asarray(field)
                    return int(arr.item())
                except Exception:
                    return None
        if 'home_race_requested' in obs:
            raw = obs['home_race_requested']
            print('home_race_requested raw:', raw)
            h = _obs_field_to_int(raw)
            print('home requested name:', race_map.get(h, 'UNKNOWN'))
        else:
            print('home_race_requested not present in obs')
        if 'away_race_requested' in obs:
            raw = obs['away_race_requested']
            print('away_race_requested raw:', raw)
            a = _obs_field_to_int(raw)
            print('away requested name:', race_map.get(a, 'UNKNOWN'))
        else:
            print('away_race_requested not present in obs')
    finally:
        try:
            env.close()
        except Exception:
            pass
except Exception as e:
    print('Demo failed:', e)
    import traceback; traceback.print_exc()


home_race_requested raw: [1]
home requested name: Terran
away_race_requested raw: [2]
away requested name: Zerg


In [19]:
# Inspect: print contents of the raw response proto (if available)
from absl import flags
import pysc2.run_configs
try:
    flags.FLAGS(['notebook', '--sc2_run_config=Windows'])
except Exception:
    pass
try:
    from pysc2.env import sc2_env
    aif = sc2_env.AgentInterfaceFormat(
        feature_dimensions=sc2_env.Dimensions(screen=64, minimap=64),
        send_observation_proto=True,
    )
    env = sc2_env.SC2Env(map_name="MoveToBeacon", players=[sc2_env.Agent(sc2_env.Race.terran)], agent_interface_format=aif, step_mul=8, game_steps_per_episode=0, visualize=False)
    try:
        timesteps = env.reset()
        obs = timesteps[0].observation
        raw = obs.get('_response_observation')
        print('_response_observation raw type:', type(raw))
        if callable(raw):
            try:
                raw = raw()
            except Exception as e:
                print('Calling raw() failed:', e)
        if raw is None:
            print('_response_observation is None')
        else:
            try:
                print('Proto object type:', type(raw))
                # try ListFields to show which top-level fields exist
                try:
                    fields = list(raw.ListFields())
                    print('ListFields count:', len(fields))
                    for fd, val in fields[:40]:
                        print(fd.name, '->', type(val))
                except Exception as e:
                    print('ListFields failed:', e)
                # show a short sample of attributes
                attrs = [f for f in dir(raw) if not f.startswith('_')]
                print('Sample attrs:', attrs[:60])
            except Exception as e:
                import traceback; traceback.print_exc()
    finally:
        try:
            env.close()
        except Exception:
            pass
except Exception as e:
    print('Inspect failed:', e)
    import traceback; traceback.print_exc()

_response_observation raw type: <class 'function'>
Proto object type: <class 's2clientprotocol.sc2api_pb2.ResponseObservation'>
ListFields count: 1
observation -> <class 's2clientprotocol.sc2api_pb2.Observation'>
Sample attrs: ['ByteSize', 'Clear', 'ClearExtension', 'ClearField', 'CopyFrom', 'DESCRIPTOR', 'DiscardUnknownFields', 'Extensions', 'FindInitializationErrors', 'FromString', 'HasExtension', 'HasField', 'IsInitialized', 'ListFields', 'MergeFrom', 'MergeFromString', 'ParseFromString', 'RegisterExtension', 'SerializePartialToString', 'SerializeToString', 'SetInParent', 'UnknownFields', 'WhichOneof', 'action_errors', 'actions', 'chat', 'observation', 'player_result']
